## Import stuff

In [42]:
import tensorflow as tf
from keras.losses import categorical_crossentropy, mean_squared_error
from tensorflow.keras import layers

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.mixture import BayesianGaussianMixture, GaussianMixture

from tqdm import tqdm
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRFClassifier, XGBRegressor, XGBRFRegressor


logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print(tf.__version__)

def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))

def get_type(model):
    return str(type(model)).split(".")[-1].split("'")[0]


2.8.0


## Load training data from csv


In [43]:
df = pd.read_csv("train_features_improved.csv")
labels_df = pd.read_csv("train_labels.csv")

X = np.array(df.iloc[:, 1:].values,  dtype=float)
Y = np.array(labels_df.iloc[:, 1:].values,  dtype=float)

#normalize
# Y[:,13] = (Y[:,13]-27)/73

# X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

print("X shape: ", X.shape)
print("Y shape: ", Y.shape)



X shape:  (18995, 513)
Y shape:  (18995, 15)


## The function that trains and rates models

In [44]:
def find_best_model(col, X, Y, X_test):
    classifiers = [
        HistGradientBoostingClassifier(),
        ExtraTreesClassifier(),
        RandomForestClassifier(),
        XGBRFClassifier(),
        XGBClassifier(),
        LGBMClassifier(),
        CatBoostClassifier(logging_level='Silent'),
    ]

    regressors = [
        HistGradientBoostingRegressor(),
        # ExtraTreesRegressor(),
        # RandomForestRegressor(),
        XGBRFRegressor(),
        XGBRegressor(),
        LGBMRegressor(),
        CatBoostRegressor(logging_level='Silent'),
    ]

    models = []

    #task 1 and 2
    if(col<11):    
        models = classifiers

    #task 3
    if(col>=11):
        models = regressors

    best_cv_score = 0
    best_model = None

    for model in models:
        start_time = time.time()
        model_type = get_type(model)
        print("training model ", model_type,"for col ", col)

        cv_score = 0
        for i in range(0,3):
            X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.9, test_size=0.1)
            Y_train = Y_train[:,col]
            Y_valid = Y_valid[:,col]

            model.fit(X_train, Y_train)
            if model_type.count("Regressor") > 0:
                pred = model.predict(X_valid)
            else:
                pred = model.predict_proba(X_valid)[:,1]
            
            if(col>=11):
                score = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y_valid, pred))
            else:
                score = metrics.roc_auc_score(Y_valid, pred)
            cv_score += score
            
        cv_score /= 3

        print("score for",model_type,": " , cv_score,", time taken: ",int(time.time() - start_time),"s")
        if cv_score > best_cv_score:
            best_cv_score = cv_score
            best_model = model

    print("best model: ", get_type(best_model),"score: ",best_cv_score,"\n")

    predictions = best_model.predict(X_test)
    return predictions,best_cv_score

## Writing results

In [45]:
X_test = pd.read_csv("test_features_improved.csv")
X_test = np.array(X_test.iloc[:, 1:].values,  dtype=float)

data = []
scores = []

for i in range(0,15):
# for i in range(0,1):
# for i in range(10,11):
# for i in range(11,15):
# for i in range(13,14):
    # if i == 11:
    #     X = np.array(pd.read_csv("train_features_imputed.csv").iloc[:, 1:].values,  dtype=float)
    #     X_test = np.array(pd.read_csv("test_features_imputed.csv").iloc[:, 1:].values,  dtype=float)

    pred,best_score = find_best_model(i, X, Y, X_test)

    pred = pred.reshape(pred.shape[0],1)
    data.append(pred)
    scores.append(best_score)





training model  HistGradientBoostingClassifier for col  0
score for HistGradientBoostingClassifier :  0.9263922159287316 , time taken:  19 s
training model  ExtraTreesClassifier for col  0
score for ExtraTreesClassifier :  0.919699830418769 , time taken:  20 s
training model  RandomForestClassifier for col  0
score for RandomForestClassifier :  0.9219606126367337 , time taken:  39 s
training model  XGBRFClassifier for col  0
score for XGBRFClassifier :  0.9247598680815733 , time taken:  19 s
training model  XGBClassifier for col  0
score for XGBClassifier :  0.9156240991324268 , time taken:  21 s
training model  LGBMClassifier for col  0
score for LGBMClassifier :  0.9280423855298651 , time taken:  6 s
training model  CatBoostClassifier for col  0
score for CatBoostClassifier :  0.9222176038183667 , time taken:  158 s
best model:  LGBMClassifier score:  0.9280423855298651 

training model  HistGradientBoostingClassifier for col  1
score for HistGradientBoostingClassifier :  0.800478997

In [46]:
print(np.mean(scores[:10]))
print(np.mean(scores[10]))
print(np.mean(scores[11:]))

print(np.mean([np.mean(scores[:10]),np.mean(scores[10]),np.mean(scores[11:])]))


0.8268128894601647
0.7264783294907655
0.7751756771899112
0.776155632046947


In [47]:
np_array = np.array(data).transpose()[0]

#un-normalize
# np_array[:,13] = (np_array[:,13]*73)+27

col_names = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2', 'LABEL_Sepsis', 'LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
print(len(col_names))
new_df = pd.DataFrame(np_array, columns=col_names)

output_csv = pd.read_csv("sample.csv")
output_csv.update(new_df)
output_csv.to_csv('submit.zip', index=False, float_format='%.3f', compression='zip')
output_csv.to_csv('submit.csv', index=False, float_format='%.3f')


15
